In [1]:
from stgraph_trainer.datasets import preprocess_data_for_lstm_model
from stgraph_trainer.datasets import inverse_diff
from stgraph_trainer.utils import get_config_from_json
from stgraph_trainer.utils import compute_metrics
from stgraph_trainer.utils import save_predictions
from stgraph_trainer.utils import save_metrics
from stgraph_trainer.models import create_lstm_model
from stgraph_trainer.trainers import LSTMTrainer
from stgraph_trainer.callbacks import PostPredictionCallback
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from functools import partial

In [6]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 135.5 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 146.5 kB/s eta 0:00:00a 0:00:01


In [2]:
# Import libraries
from stgraph_trainer.datasets import load_province_temporal_data
from stgraph_trainer.datasets import data_diff
from stgraph_trainer.datasets import timeseries_to_supervised
from stgraph_trainer.datasets import preprocess_data_for_lstm_model
from stgraph_trainer.datasets import inverse_diff
from stgraph_trainer.utils import get_config_from_json
from stgraph_trainer.utils import compute_metrics
from stgraph_trainer.utils import save_predictions
from stgraph_trainer.utils import save_metrics
from stgraph_trainer.models import create_lstm_model
from stgraph_trainer.trainers import LSTMTrainer
from stgraph_trainer.callbacks import PostPredictionCallback
import os
import tensorflow as tf
import numpy as np
from functools import partial

In [3]:
# Set up configs and parameters
data_config_file = os.getcwd() + '/configs/data_config.json'
lstm_config_file = os.getcwd() + '/configs/lstm_config.json'
model_name = 'lstm'
work_dir = os.getcwd() + '/results'

data_configs = get_config_from_json(data_config_file)
lstm_configs = get_config_from_json(lstm_config_file)

In [4]:
data_configs = get_config_from_json(data_config_file)
lstm_configs = get_config_from_json(lstm_config_file)

In [5]:
PROVINCES = data_configs['provinces']
SPLIT_DATE = data_configs['split_date']
TIME_STEPS = int(data_configs['time_steps'])
STATUS = data_configs['status']

In [6]:
LSTM_UNITS = int(lstm_configs['lstm_units'])
BATCH_SIZE = int(lstm_configs['batch_size'])
DROP_RATE = float(lstm_configs['drop_rate'])
RECURRENT_DROP_RATE = float(lstm_configs['recurrent_drop_rate'])
TRIALS = int(lstm_configs['trials'])
EPOCHS = int(lstm_configs['epochs'])

In [7]:
# Load and process dataset
df = pd.read_csv('../notebooks/stgraph_trainer/datasets/data/water_nitrite (1).csv', index_col=0)
df = df.set_index('dateTime')


#df = load_province_temporal_data(provinces=PROVINCES, status=STATUS)
#df = pd.read_csv('../notebooks/stgraph_trainer/datasets/data/temporal_data.csv')
#df = df.set_index('datetime')
X_train, y_train, X_test, y_test, _, _, scaler = preprocess_data_for_lstm_model(df,
                                                                                SPLIT_DATE,
                                                                                TIME_STEPS)


In [8]:

n_features = X_train.shape[-1]
n_test_samples = len(y_test)
output_size = X_train.shape[-1]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

# Functions to han

(267, 7, 24) (267, 24) (91, 7, 24) (91, 24)


In [9]:
# Functions to handle post-prediction
# including inverse transform and add back difference
# to make predictions back to original scale.
def inverse_transform(x, scaler, **kwargs):
  return scaler.inverse_transform(x)

inv_trans = partial(inverse_transform, scaler=scaler)
inv_trans._order = 10

In [10]:
def inv_diff_1(x, raw_values, n_test=None, idx=0):
  return inverse_diff(x, raw_values, n_test + 1 - idx)
inv_diff = partial(inv_diff_1, raw_values=df.values, n_test=n_test_samples)
inv_diff._order = 20

tfms = [inv_trans, inv_diff]

In [23]:
n_features

24

In [11]:
for trial in range(TRIALS):
  model = create_lstm_model(LSTM_UNITS,
                            output_size,
                            BATCH_SIZE,
                            TIME_STEPS,
                            n_features,
                            DROP_RATE,
                            RECURRENT_DROP_RATE)
  print(model.summary())

ValueError: Unrecognized keyword arguments passed to LSTM: {'batch_shape': (1, 7, 24)}

In [19]:
# Create model, train and evaluate
rmse_results = []
mae_results = []

for trial in range(TRIALS):
  model = create_lstm_model(LSTM_UNITS,
                            output_size,
                            BATCH_SIZE,
                            TIME_STEPS,
                            n_features,
                            DROP_RATE,
                            RECURRENT_DROP_RATE)
  print(model.summary())
  train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
  test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)

  loss_func = tf.losses.MeanSquaredError()
  optimizer = tf.optimizers.Adam(learning_rate=0.01)

  trainer = LSTMTrainer(model,
                        train_ds,
                        test_ds,
                        loss_func,
                        optimizer,
                        callbacks=[PostPredictionCallback(funcs=tfms)],
                        raw_test=df.iloc[-n_test_samples:].values)

  history = trainer.train(EPOCHS)
  print(history)

  predict = trainer.predict()
  save_predictions(predict,
                   model_name,
                   n_exp=trial,
                   columns=PROVINCES,
                   index=df.iloc[-n_test_samples:].index,
                   path=work_dir)
  print(predict.shape)

  m, m_avg = compute_metrics(df.iloc[-n_test_samples:], predict, metric='rmse')
  m = np.append(m, m_avg)
  rmse_results.append(m)
  print(m, m_avg)
  
  m, m_avg = compute_metrics(df.iloc[-n_test_samples:], predict, metric='mae')
  m = np.append(m, m_avg)
  mae_results.append(m)
  print(m, m_avg)


ValueError: Unrecognized keyword arguments passed to LSTM: {'batch_input_shape': (1, 7, 24)}

!python3 --version

In [9]:
!pip3 install --ignore-installed --upgrade https://storage.googleapis.com/tensorflow/windows/cpu/tensorflow-1.2.1-cp36-cp36m-win_amd64.whl

tensorflow-1.2.1-cp36-cp36m-win_amd64.whl is not a supported wheel on this platform.
You are using pip version 18.1, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
!pip3 install --ignore-installed --upgrade tensorflow_io_gcs_filesystem-0.23.1-cp36-cp36m-manylinux_2_17_aarch64.manylinux2014_aarch64.whl


ERROR: tensorflow_io_gcs_filesystem-0.23.1-cp36-cp36m-manylinux_2_17_aarch64.manylinux2014_aarch64.whl is not a supported wheel on this platform.


In [1]:
!pip install tensorflow